# Analysing historical records and past models

Access to historical records and past models is important to 
* evaluate global and local impacts of climate change & anthropogenic influences and 
* test and validate models. In the modelling area, this kind of approach is done  routinely to evaluate models performances (hindcast models).

Here we will see how we can query historical data from a buoy located offshore of Sydney and how to get wave historical data from the WaveWatch3 model for Maroubra Beach (NSW-Australia). 

As for last notebbok, we will need the **netCDF** libray (<a ref='http://www.unidata.ucar.edu/netcdf/'>Network Common Data Form</a>) is a set of software libraries and self-describing, machine-independent data formats that support the creation, access, and sharing of array-oriented scientific data. The project homepage is hosted by the **Unidata** program at the University Corporation for Atmospheric Research (**UCAR**).

Loading a module is straight forward:

In [ ]:
%matplotlib inline
from pylab import *
import netCDF4
import datetime as dt

# Historical buoy data

We will use a dataset containing ocean wave data from a wave monitoring buoy moored off Sydney at latitude -33.775278, longitude 151.417778 and water depth 85 metres.  

The data is gathered using the Directional Waverider system developed by the Dutch company, Datawell. The Directional Waverider buoy utilises a heave-pitch-roll sensor, two fixed X and Y accelerometers and a three axis fluxgate compass to measure both vertical and horizontal motion.  An on-board processor converts the buoy motion to three orthogonal (vertical, north-south, east-west) translation signals that are transmitted to the shore station.  The directional spectrum is also routinely transmitted to the receiving station for further processing.  The wave data is stored on the receiving station PC before routine transfer to Manly Hydraulics Laboratory.

In [ ]:
# Offshore Sydney buoy data
nc_data=netCDF4.Dataset('http://www.metoc.gov.au/thredds/dodsC/MHLWAVE/Sydney/IMOS_ANMN-NSW_W_20050215T020000Z_WAVESYD_FV01_END-20080312T210000Z.nc')

## Query dataset

Let have a look at the loaded netCDF variables

In [ ]:
nc_data.variables.keys()

In [ ]:
print nc_data.variables['HRMS']

## Find out time interval record

Get the time extension of the gathered data...

In [ ]:
start = dt.datetime(1985,1,1)
# Get desired time step  
time_var = nc_data.variables['TIME']
itime = netCDF4.date2index(start,time_var,select='nearest')
dtime = netCDF4.num2date(time_var[itime],time_var.units)
daystr = dtime.strftime('%Y-%b-%d %H:%M')
print 'buoy record start time:',daystr

In [ ]:
end = dt.datetime(2018,1,1)
# Get desired time step  
time_var = nc_data.variables['TIME']
itime2 = netCDF4.date2index(end,time_var,select='nearest')
dtime2 = netCDF4.num2date(time_var[itime2],time_var.units)
dayend = dtime2.strftime('%Y-%b-%d %H:%M')
print 'buoy record end time:',dayend

## Buoy location

Check the location of the data

In [ ]:
loni = nc_data.variables['LONGITUDE'][:]
lati = nc_data.variables['LATITUDE'][:]
print loni,lati
names=[]
names.append('Offshore Sydney Buoy')

## Visualise buoy records

Export for the hostorical time serie, the desired buoy variable, here I've used the mean wave height:

In [ ]:
times = nc_data.variables['TIME']
jd_data = netCDF4.num2date(times[:],times.units).flatten()
hm_data = nc_data.variables['HMEAN'][:].flatten()
#hs_data = ma.masked_where(hs_data > 98., hs_data)

Now plot the exported dataset

In [ ]:
# make the time series plot, with nicely formatted labels
MyDateFormatter = DateFormatter('%Y-%b-%d')
fig = plt.figure(figsize=(8,6), dpi=80) 
ax1 = fig.add_subplot(111)

ax1.plot(jd_data[itime:itime2],hm_data[itime:itime2]) 
ax1.xaxis.set_major_locator(WeekdayLocator(byweekday=MO,interval=12))
ax1.xaxis.set_major_formatter(MyDateFormatter)
ax1.grid(True)
setp(gca().get_xticklabels(), rotation=45, horizontalalignment='right')
plt.title('IMOS Offshore Sydney Buoy Data')
ax1.set_ylabel('meters')
ax1.legend(names,loc='upper right')

# Query historic model data for a specific location

We use data from WW3 model. 
Description of the Wavewatch3 model can be found <a href='http://polar.ncep.noaa.gov/waves/wavewatch/'>here</a>.

<img src='http://www.worldwindsinc.com/wp-content/uploads/2012/12/wavewatch.png', width=600>

# Loading historical model output

Load the netCDF data from the wavewatch-3 model and query the variables which are embedded in the dataset

In [ ]:
#OCEAN FORECAST MODEL WW3
nc_data=netCDF4.Dataset('http://134.178.63.198/thredds/dodsC/paccsapwaves_gridded/ww3.glob_24m.199801.nc')

In [ ]:
nc_data.variables.keys()

In [ ]:
print nc_data.variables['t']

## Find out time interval record

Get the time extension of the gathered data

In [ ]:
start = dt.datetime(1980,1,1)
# Get desired time step  
time_var = nc_data.variables['time']
itime = netCDF4.date2index(start,time_var,select='nearest')
dtime = netCDF4.num2date(time_var[itime],time_var.units)
daystart = dtime.strftime('%Y-%b-%d %H:%M')
print 'WW3 dataset start time:',daystart

In [ ]:
end = dt.datetime(2018,1,1)
# Get desired time step  
time_var = nc_data.variables['time']
itime2 = netCDF4.date2index(end,time_var,select='nearest')
dtime2 = netCDF4.num2date(time_var[itime2],time_var.units)
dayend = dtime2.strftime('%Y-%b-%d %H:%M')
print 'WW3 dataset end time:', dayend

## Find historical WW3 prediction for a specific localtion

Define the longitude and latitude of Maroubra Beach

In [ ]:
maroubra_lon=[151.25750340000002]
maroubra_lat=[-33.947314899999]

In [ ]:
# Function to find index to nearest point
def near(array,value):
    idx=(abs(array-value)).argmin()
    return idx

Find the row and column number corresponding to the nearest WW3 model grid point to the requested location

In [ ]:
lon_model=nc_data.variables['longitude'][:].flatten()
lat_model=nc_data.variables['latitude'][:].flatten()
ix = near(lon_model, maroubra_lon)
iy = near(lat_model, maroubra_lat)
print ix, iy

### Extract WW3 data for Maroubra beach

Extract the significant wave height from the WW3 model for this particular point

In [ ]:
times = nc_data.variables['time']
jd_model = netCDF4.num2date(times[:],times.units)
hs_model = nc_data.variables['hs'][:,ix,iy]
istart_model = netCDF4.date2index(start,times,select='nearest')
istop_model = netCDF4.date2index(end,times,select='nearest')

We have everything to plot the time serie:

In [ ]:
# make the time series plot, with nicely formatted labels
MyDateFormatter = DateFormatter('%Y-%b-%d')
fig = plt.figure(figsize=(5,5), dpi=80) 
ax1 = fig.add_subplot(111)

names=[]
names.append('Maroubra')

#ax1.plot(jd_data[istart:istop],hs_data[istart:istop]) 
ax1.plot(jd_model[istart_model:istop_model],hs_model[istart_model:istop_model])
#ax1.plot(jd_mod2[istart_mod2:istop_mod2],hs_mod2[istart_mod2:istop_mod2])
ax1.xaxis.set_major_locator(WeekdayLocator(byweekday=MO))
ax1.xaxis.set_major_formatter(MyDateFormatter)
ax1.grid(True)
setp(gca().get_xticklabels(), rotation=45, horizontalalignment='right')
plt.title('WAVEWATCH III model')
ax1.set_ylabel('meters')
ax1.legend(names,loc='upper right')